<a href="https://colab.research.google.com/github/ymkge/competiton/blob/main/pytorch_fakenews_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 参考notebook
https://www.kaggle.com/code/debarshichanda/pytorch-w-b-jigsaw-starter

In [ ]:
!nvidia-smi

Mon Mar 28 09:53:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install --upgrade wandb

     |████████████████████████████████| 1.7 MB 15.2 MB/s 
     |████████████████████████████████| 144 kB 88.8 MB/s 
     |████████████████████████████████| 181 kB 83.1 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=ffe2b439c9b86e85e1bd93729f9a030fe162eaacf16c3f93d94d789a40869adc
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
!pip install GPUtil
!pip install transformers==4.12.2
!pip install fugashi
!pip install mecab-python3
!pip install ipadic

!pip install colorama

  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=b54a82fadadaf62f11348bb7087b45104d4e81ae7db6b25b82d09ba905c3df57
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
     |████████████████████████████████| 3.1 MB 14.1 MB/s 
     |████████████████████████████████| 3.3 MB 75.3 MB/s 
     |████████████████████████████████| 596 kB 83.1 MB/s 
     |████████████████████████████████| 895 kB 72.7 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 568 kB 14.2 MB/s 
     |████████████████████████████████| 574 kB 14.5 MB/s 
     |████████████████████████████████| 13.4 MB 14.7 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=0ca0dacdddcb2f72ee

In [ ]:
import os
import gc
import copy
import time
import random
import string

import datetime
from datetime import datetime, timedelta, timezone

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pickle
import re
import unicodedata

import GPUtil
import regex
import scipy as sp
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, AdamW
  
warnings.filterwarnings('ignore')

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
def id_generator(size=12, chars=string.ascii_lowercase + string.digits):
    return ''.join(random.SystemRandom().choice(chars) for _ in range(size))

JST = timezone(timedelta(hours=+9), 'JST')
now = datetime.now(JST)
current_time = now.strftime("%Y-%m-%d-%H-%M", )

HASH_NAME = "Fakenews-"+current_time

print(HASH_NAME)

Fakenews-2022-03-28-18-54


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT = "/content/drive/MyDrive/python/cpt-fakenews"
CUR_DIR = ROOT+"/baseline/code"
%cd $CUR_DIR

/content/drive/MyDrive/python/cpt-fakenews/baseline/code


In [ ]:
CONFIG = {"seed": 2022,
          "epochs": 20,
          "model_name": r'cl-tohoku/bert-base-japanese-whole-word-masking',  # r'cl-tohoku/bert-base-japanese-v2'
          "train_batch_size": 32,
          "valid_batch_size": 64,
          "max_length": 128,
          "learning_rate": 1e-6,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-7,
          "T_max": 500,
          "weight_decay": 1e-6,
          "max_grad_norm" :1.0,
          "n_accumulate": 1,
          "num_classes": 2,
          "n_fold": 5,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "hash_name": HASH_NAME,
          "hidden_size" :768,
          "num_hidden_layers": 24,    
          "dropout" :0.2
          }

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])
CONFIG['group'] = f'{HASH_NAME}-Baseline'

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [ ]:
df = pd.read_csv(ROOT+"/data/train.csv")
df

,id,isFake,text
0,2877,0,サッカーJ2リーグの東京ヴェルディは6月28日（UTC+9 以下同）、東京ヴェルディホールデ...
1,6827,1,朝日新聞など各報道によると、イギリスに本拠を置く携帯電話世界最大手ボーダフォンの日本法人(ソ...
2,518,0,9月の総選挙の結果を受け、「大連立」を組んだドイツの二大会派、ドイツキリスト教民主同盟（CD...
3,1670,0,四国新聞によると、プロ野球選手だった野口二郎さんが5月21日、肺炎のため宝塚市の病院で死去し...
4,4710,1,ロイターなど各社の報道によると、タイ王国で21日(C+7)、元反政府活動家7人が政府の転覆を...
...,...,...,...
3776,3092,0,朝日放送によると、11月24日夜7:00から毎日放送・MBSテレビ局で放送する予定だった大阪...
3777,3772,1,毎日新聞と北海道新聞によると、民主党の北海道5区で立候補した民主党の山本広和被告(60歳)が...
3778,5191,1,朝日新聞によると、インドネシアで、10月1日からガソリンや灯油などが値上げされる。2009年...
3779,5226,1,朝日新聞によると、14日午後0時40分頃、大阪市北区梅田2丁目の阪神百貨店の『・(阪神・阪急...


In [ ]:
df["isFake"].value_counts()

1    1937
0    1844
Name: isFake, dtype: int64

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Create Folds</h1></span>

In [ ]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'], shuffle=True, random_state=CONFIG['seed'])

for fold, ( _, val_) in enumerate(skf.split(X=df, y=df["isFake"])):
    df.loc[val_ , "kfold"] = int(fold)
    
df["kfold"] = df["kfold"].astype(int)
df.head()

,id,isFake,text,kfold
0,2877,0,サッカーJ2リーグの東京ヴェルディは6月28日（UTC+9 以下同）、東京ヴェルディホールデ...,0
1,6827,1,朝日新聞など各報道によると、イギリスに本拠を置く携帯電話世界最大手ボーダフォンの日本法人(ソ...,3
2,518,0,9月の総選挙の結果を受け、「大連立」を組んだドイツの二大会派、ドイツキリスト教民主同盟（CD...,4
3,1670,0,四国新聞によると、プロ野球選手だった野口二郎さんが5月21日、肺炎のため宝塚市の病院で死去し...,3
4,4710,1,ロイターなど各社の報道によると、タイ王国で21日(C+7)、元反政府活動家7人が政府の転覆を...,1


In [ ]:
class FakenewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['text'].values
        self.target = df['isFake'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
    
        text = self.text[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
    
        target = self.target[index]
        
        onehot_t = np.zeros(CONFIG["num_classes"], dtype=np.float32) + 0.0025 ### Label smoothing
        onehot_t[target] = 0.995
        
        ids = inputs_text['input_ids']
        mask = inputs_text['attention_mask']
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'target': torch.tensor(onehot_t, dtype=torch.float)
        }

In [ ]:
class FakenewsModel(nn.Module):
    def __init__(self, model_name):
        super(FakenewsModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, CONFIG['num_classes'])
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        outputs = self.sigmoid(outputs)
        
        return outputs.squeeze()

In [ ]:
def criterion(outputs, targets):
    loss_f = nn.BCELoss()
    """
    print("output",outputs.shape, 'targets',targets.shape)
    print(outputs)
    print(targets)
    """
    return loss_f(outputs,targets)

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = ids.size(0)

        outputs = model(ids, mask)
        
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = ids.size(0)

        outputs = model(ids, mask)
        
        loss = criterion(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold, savepath):
    # To automatically log gradients
    wandb.watch(model, log_freq=100)
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
        # Log the metrics
        wandb.log({"Train Loss": train_epoch_loss})
        wandb.log({"Valid Loss": val_epoch_loss})
        
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"{b_}Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            run.summary["Best Loss"] = best_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            #PATH = f"Loss-Fold-{fold}.bin"    ####ここのPATHは変えよう
            PATH = savepath + f"Loss-Fold-{fold}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print(f"Model Saved{sr_}")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
def prepare_loaders(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = FakenewsDataset(df_train, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])
    valid_dataset = FakenewsDataset(df_valid, tokenizer=CONFIG['tokenizer'], max_length=CONFIG['max_length'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [ ]:

modelpath = f"{ROOT}/baseline/models/bert-{current_time}/"  
if not os.path.isdir(modelpath):
    os.makedirs(modelpath)

for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    run = wandb.init(project='Fakenews', 
                     config=CONFIG,
                     job_type='Train',
                     group=CONFIG['group'],
                     tags=['bert', f'{HASH_NAME}', 'bceloss'],
                     name=f'{HASH_NAME}-fold-{fold}',
                     anonymous='must')
    
    # Create Dataloaders
    train_loader, valid_loader = prepare_loaders(fold=fold)
    
    model = FakenewsModel(CONFIG['model_name'])
    model.to(CONFIG['device'])
    
    # Define Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    
    model, history = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=fold,
                                  savepath=modelpath)
    
    run.finish()
    
    del model, history, train_loader, valid_loader
    _ = gc.collect()
    print()

wandb: Currently logged in as: chi_naka (use `wandb login --relogin` to force relogin)


====== Fold: 0 ======


Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Using GPU: Tesla V100-SXM2-16GB



...

100%|██████████| 12/12 [00:02<00:00,  5.74it/s, Epoch=19, LR=6.51e-7, Valid_Loss=0.256]


Validation Loss Improved (0.2686579634272863 ---> 0.25640144344990845)
Model Saved



100%|██████████| 12/12 [00:02<00:00,  5.82it/s, Epoch=20, LR=8.78e-7, Valid_Loss=0.243]


Validation Loss Improved (0.25640144344990845 ---> 0.24327793811994886)
Model Saved

Training complete in 0h 10m 13s
Best Loss: 0.2433



Train Loss,███▇▇▇▇▆▆▄▃▃▂▂▁▁▁▁▁▁
Valid Loss,██▇▇▇▇▆▆▅▄▂▂▂▂▁▁▁▁▁▁
Best Loss,0.24328
Train Loss,0.17619
Valid Loss,0.24328


## Validation

In [ ]:
!ls /content/drive/MyDrive/python/cpt-fakenews/baseline/models/

bert-2022-03-03-05-38  bert-2022-03-11-12-21  bert-2022-03-18-16-52
bert-2022-03-11-00-10  bert-2022-03-18-15-22  bert-2022-03-28-18-54


In [ ]:
from glob import glob
MODEL_PATHS = glob(f'{modelpath}/*.bin')
MODEL_PATHS.sort()

In [ ]:
MODEL_PATHS

['/content/drive/MyDrive/python/cpt-fakenews/baseline/models/bert-2022-03-28-18-54/Loss-Fold-0.bin',
 '/content/drive/MyDrive/python/cpt-fakenews/baseline/models/bert-2022-03-28-18-54/Loss-Fold-1.bin',
 '/content/drive/MyDrive/python/cpt-fakenews/baseline/models/bert-2022-03-28-18-54/Loss-Fold-2.bin',
 '/content/drive/MyDrive/python/cpt-fakenews/baseline/models/bert-2022-03-28-18-54/Loss-Fold-3.bin',
 '/content/drive/MyDrive/python/cpt-fakenews/baseline/models/bert-2022-03-28-18-54/Loss-Fold-4.bin']

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        
        #PREDS.append(outputs.view(-1).cpu().detach().numpy()) 
        PREDS.append(outputs.cpu().detach().numpy()) 

    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = FakenewsModel(CONFIG['model_name'])
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [ ]:
def get_labels_frompred(out, multi=False):

    print(out)
    o = (-out).argsort(1) 
    preds = int(o[0][0].cpu().numpy())
    print(preds)

    return preds

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

fold_f1scores = []
fold_accuracyscores = []

for fold in range(0, CONFIG['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    
    train_loader, valid_loader = prepare_loaders(fold=fold)

    valid = df[df["kfold"]==fold]

    out = inference([MODEL_PATHS[fold]], valid_loader, CONFIG['device'])

    valid["preds"] = np.argmax(out, axis=1)

    fold_f1scores.append(f1_score(valid["isFake"].values, valid["preds"].values))
    fold_accuracyscores.append(accuracy_score(valid["isFake"].values, valid["preds"].values))


====== Fold: 0 ======


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 12/12 [00:02<00:00,  5.66it/s]


====== Fold: 1 ======


...

100%|██████████| 12/12 [00:01<00:00,  6.01it/s]


====== Fold: 4 ======


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 12/12 [00:02<00:00,  5.87it/s]


In [ ]:
fold_f1scores

[0.9278350515463918,
 0.9046979865771811,
 0.9174560216508795,
 0.9090909090909091,
 0.9171122994652405]

In [ ]:
fold_accuracyscores

[0.9260237780713342,
 0.906084656084656,
 0.9193121693121693,
 0.9087301587301587,
 0.917989417989418]